In [1]:
# imports
import ann_cuda as nn
import numpy as np
from sklearn.datasets import load_digits

In [2]:
# global variables
batch_size = 10
learning_rate = 0.02
epochs = 1000
load = True
save = True
directory = ''

In [3]:
# load mnist dataset from sklearn
X, Y = (load_digits()['data'], load_digits()['target'])
X = X.reshape(-1, 1, 8, 8).astype(np.float32)

In [4]:
# one-hot encode
Y_enc = np.zeros((Y.size, 10, 1, 1), dtype=np.float32)
for i in range(Y.size):
    for x in range(10):
        if Y[i] == x:
            Y_enc[i,x] = 1

In [5]:
# split dataset in to batches
Y_enc2 = Y_enc.copy()
X_batched = np.array_split(X, X.shape[0]/batch_size, axis=0)
Y_batched = np.array_split(Y_enc, X.shape[0]/batch_size, axis=0)
Y_batched2 =np.array_split(Y_enc2, X.shape[0]/batch_size, axis=0)
for i in range(len(X_batched)):
    if X_batched[i].shape[0] != batch_size:
        X_batched[i] = X_batched[i][:batch_size, :, :, :]
        Y_batched[i] = Y_batched[i][:batch_size, :, :, :]
        Y_batched2[i] = Y_batched2[i][:batch_size, :, :, :]

In [6]:
# initialize network
net = nn.Network()

# set up network architecture
input = net + nn.Input("input1")([1, 8, 8])
x = net + nn.Conv2D("conv1", 20, 5, 1, 0, 1)(input)
x = net + nn.Activation("relu1", nn.CUDNN_ACTIVATION_RELU, 0)(x)
x = net + nn.Pooling("pool1", 2, 0, 2, nn.CUDNN_POOLING_MAX)(x)
x = net + nn.Dense("dense_1", 200)(x)
x = net + nn.Activation("relu2", nn.CUDNN_ACTIVATION_RELU, 0)(x)
x = net + nn.Dense("dense2", 10)(x)
out = net + nn.Softmax("softmax1")(x)

# load pretrained model weights from specified directory
if (load):
    net.load(directory)

net.cuda() # move to device, topologically sort layer graph to avoid dependency issues, and then find output layers
net.train() # set network to train

...Loading Weights...

...Model Configuration...

CUDA: input1 

CUDA: conv1 

CUDA: relu1 

CUDA: pool1 

CUDA: dense_1 

CUDA: relu2 

CUDA: dense2 

CUDA: softmax1 



In [7]:
# view topological sorting of layers
net.layers_topological()

[input1	Shape: { 1, 8, 8 },
 conv1	Output Channels: 20,
 relu1	Activation Mode: 1,
 pool1	Pooling Mode: 0,
 dense_1	Output Size: { 200 },
 relu2	Activation Mode: 1,
 dense2	Output Size: { 10 },
 softmax1]

In [8]:
# view output layers
net.output_layers()

[softmax1]

In [9]:
# convert batched numpy arrays to Tensor instances and then move to device
for i, zipped in enumerate(zip(X_batched, Y_batched, Y_batched2)):
    X_batched[i] = nn.Tensor(zipped[0])
    X_batched[i].to(nn.cuda)
    Y_batched[i] = nn.Tensor(zipped[1])
    Y_batched[i].to(nn.cuda)
    Y_batched2[i] = nn.Tensor(zipped[2])
    Y_batched2[i].to(nn.cuda)

In [10]:
# train network
net.train()
for epoch in range(epochs):
    for batch, labels in zip(X_batched[30:], Y_batched[30:]):
        # tensor = nn.Tensor(batch)
        # tensor.to(nn.cuda)
        # target = nn.Tensor(labels)
        # target.to(nn.cuda)
        
        net.forward([batch])
        net.backward([labels])
        net.update(learning_rate)
    #print(f"Epoch {epoch}: {net.get_accuracy([target])[0] * 100 / batch_size}% accuracy")

..loaded  conv1  pretrained weights and biases..

..loaded  dense_1  pretrained weights and biases..

..loaded  dense2  pretrained weights and biases..



In [11]:
# test network
net.test()
tp_count = [0, 0]
step = 0
for batch, labels in zip(X_batched[:30], Y_batched[:30]):
	net.forward([batch])
	tp_count[0] += net.get_accuracy([labels])[0]

	# fetch next data
	step += 1
loss = net.loss([labels])
accuracy1 = 100 * tp_count[0] / step / batch_size

print(f"Loss1: {loss[0]}, Accuracy1: {accuracy1}%")

Loss1: 0.0, Accuracy1: 94.66666666666666%


In [12]:
# save network weights to specified directory
if (save):
    net.save(directory)

...Storing Weights...

..saving input1 weights and biases..

..done..

..saving conv1 weights and biases..

..done..

..saving relu1 weights and biases..

..done..

..saving pool1 weights and biases..

..done..

..saving dense_1 weights and biases..

..done..

..saving relu2 weights and biases..

..done..

..saving dense2 weights and biases..

..done..

..saving softmax1 weights and biases..

..done..

